In [91]:
import requests
from bs4 import BeautifulSoup
import plotly.express as px
import pandas as pd

github = 'https://github.com'
URL = 'https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports_us'

page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
# find the latest report
latest = str(soup.table.find_all('a')[-4]['href'])
# print(latest)
# go to the latest report page
latestURL = github + latest
latestPage = requests.get(latestURL, 'html.parser')
# print(latestURL)
# find the raw page url
soup = BeautifulSoup(latestPage.content, 'html.parser')
raw = str(soup.find(id='raw-url')['href'])
rawURL = github + raw
# print(rawURL)
df = pd.read_csv(rawURL)
# print(df.head(20))
df = df.dropna(axis=0, subset=['Lat'])
df = df.fillna(0)
# display(df.head())
date, time = df.iloc[0, 2].split()
# print(date)
px.set_mapbox_access_token(open(".mapbox_token").read())

fig = px.scatter_mapbox(df,
                        lat="Lat",
                        lon="Long_",
                        color="Deaths", size="Deaths",
                        color_continuous_scale=px.colors.cyclical.IceFire,
                        size_max=40,
                        zoom=3,
                        hover_data=["Province_State", "Confirmed"]
                        )

fig.update_layout(
    title=f'COVID-19 Deaths in USA by states. Date: {date}',
)
fig.show()